In [ ]:
!pip install opencv-contrib-python==3.4.2.17

import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold   
import keras
from keras.datasets import cifar10
from scipy.cluster.vq import *
import sklearn.metrics as metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
def desSIFT(image):
    sift = cv2.xfeatures2d.SIFT_create()
    kp, des = sift.detectAndCompute(image,None)	
    #img2 = cv2.drawKeypoints(img,kp,None,(255,0,0),4)
    #plt.imshow(img2),plt.show()
    return kp,des

def describeSURF( image):
    surf = cv2.xfeatures2d.SURF_create()
    kp, des = surf.detectAndCompute(image,None)
    return kp,des


def getDescriptors(images, labels_g) : 
    descriptors = []
    labels = []
    count = 0
    
    print (images.shape)
    for image in images : 
        print (image.shape)
        #Converting the image into grayscale         
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
        #Re-sizing the image
        image = cv2.resize(image, (150, 150), interpolation=cv2.INTER_AREA)
        kp, des = describeSURF(image)
        
        if des is not None : 
            print (des.shape)
            descriptors.append(des)
            labels.append(int(labels_g[count]))
        count += 1
            
    
    print (len(labels))
    
    descriptorsFin = descriptors[0]
    
    for descriptor in descriptors[1:]:
        if descriptor is not None:
            descriptorsFin = np.vstack((descriptorsFin, descriptor))
    
        
    return descriptorsFin, descriptors, labels

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
sift_des, descriptors, sift_labels = getDescriptors(np.concatenate((x_train, x_test), axis = 0), np.concatenate((y_train, y_test)))



Streaming output truncated to the last 5000 lines.
(82, 64)
(32, 32, 3)
(100, 64)
(32, 32, 3)
(86, 64)
(32, 32, 3)
(39, 64)
(32, 32, 3)
(78, 64)
(32, 32, 3)
(108, 64)
(32, 32, 3)
(68, 64)
(32, 32, 3)
(77, 64)
(32, 32, 3)
(56, 64)
(32, 32, 3)
(56, 64)
(32, 32, 3)
(85, 64)
(32, 32, 3)
(48, 64)
(32, 32, 3)
(100, 64)
(32, 32, 3)
(61, 64)
(32, 32, 3)
(77, 64)
(32, 32, 3)
(91, 64)
(32, 32, 3)
(94, 64)
(32, 32, 3)
(58, 64)
(32, 32, 3)
(39, 64)
(32, 32, 3)
(77, 64)
(32, 32, 3)
(67, 64)
(32, 32, 3)
(56, 64)
(32, 32, 3)
(93, 64)
(32, 32, 3)
(51, 64)
(32, 32, 3)
(60, 64)
(32, 32, 3)
(61, 64)
(32, 32, 3)
(102, 64)
(32, 32, 3)
(86, 64)
(32, 32, 3)
(86, 64)
(32, 32, 3)
(58, 64)
(32, 32, 3)
(54, 64)
(32, 32, 3)
(64, 64)
(32, 32, 3)
(94, 64)
(32, 32, 3)
(36, 64)
(32, 32, 3)
(113, 64)
(32, 32, 3)
(84, 64)
(32, 32, 3)
(81, 64)
(32, 32, 3)
(77, 64)
(32, 32, 3)
(59, 64)
(32, 32, 3)
(59, 64)
(32, 32, 3)
(52, 64)
(32, 32, 3)
(54, 64)
(32, 32, 3)
(79, 64)
(32, 32, 3)
(94, 64)
(32, 32, 3)
(13, 64)
(32, 32, 3)

In [5]:
k = 60
voc,  variance = kmeans((sift_des), k, 1)
print (np.concatenate((y_train, y_test)).shape)

(60000, 1)


In [ ]:
print (len(sift_labels))
print (len(sift_des))

# Constructing a histogram of k clusters and number of images having those clusters
imageFeatures = np.zeros((len(sift_labels), k), "float32")
for i in range(len(sift_labels)):
    words, distance = vq(descriptors[i],voc)
    for w in words:
        imageFeatures[i][w] += 1

In [7]:
# Normalizing the histogram(Image Features)
stdSlr = StandardScaler().fit(imageFeatures)
imageFeatures = stdSlr.transform(imageFeatures)

In [ ]:
# Splitting the Image Features into test and train
X_train, X_test, y_train, y_test = train_test_split(imageFeatures, sift_labels, test_size=0.1, random_state=4)

In [9]:
# Training the train data using knn
clf = cv2.ml.KNearest_create()
clf.train(X_train, cv2.ml.ROW_SAMPLE, np.asarray(y_train, dtype=np.float32))

True

In [10]:
# Predicting the test values
ret, results, neighbours ,dist = clf.findNearest(X_test, k=10)

In [11]:
pred_label = []
for var in results:
    label = var
    pred_label.append(int(label))
    
#accuracies
metrics.accuracy_score(y_test, pred_label)

0.24933333333333332

0.24933333333333332